In [18]:
from collections import namedtuple

In [118]:
class System:
    _k1 = 1.34
    _k2 = 1.6E9
    _k3 = 8000
    _k4 = 4E7
    _k5 = 0.5
    _f = 1.45
    _a = 0.06
    F = namedtuple('F' , 'x y z') 
    
    def F_init(self,x,y,z):
        #dX/dt
        self.F.x = self._k1*self._a*y-self._k2*x*y+self._k3*self._a*x-2*self._k4*x**2
        #dY/dt
        self.F.y = -self._k1*self._a*y-self._k2*x*y+self._f*self._k5*z
        #dZ/dt
        self.F.z = self._k3*self._a*x-self._k5*z
        
    def F_derivative_init(self,x,y,z):
        F_derivative = [
            [-self._k2*y+self._k3*self._a-4*self._k4*x,self._k1*self._a-self._k2*x, 0],
            [-self._k2*y, -self._k1*self._a-self._k2*x, self._f*self._k5],
            [self._a*self._k3, 0, -self._k5]
                            ]
        return F_derivative
    
    def Gauss(self,matrix, dim):
        x = [0 for i in range(dim)]
        for j in range(dim):
            
            ind = j
            maximum = matrix[j][j]
            for i in range(j+1,dim):
                if (matrix[i][j] > maximum):
                    maximum = matrix[i][j]
                    ind = i
            matrix[ind],matrix[j]=matrix[j],matrix[ind]
            
            for i in range (j+1,dim):
                d = -matrix[i][j]/matrix[j][j]
                for j1 in range(dim+1):
                    matrix[i][j1]+=matrix[j][j1]*d
                    
        for i in reversed(range(dim)):
            summ = matrix[i][dim]
            for j in reversed(range(dim)):
                summ-=matrix[i][j]*x[j]
            x[i] = summ/matrix[i][i]
        return x
    
    def Newthon(self,x,y,z): 
        der = self.F_derivative_init(x,y,z)
        der[0].append(-self.F.x)
        der[1].append(-self.F.y)
        der[2].append(-self.F.z)
        #print(der)
        x0 = self.Gauss(der,len(der))
    def __init__(self,x,y,z):
        self.F_init(x,y,z)
        self.F_derivative_init(x,y,z)

In [119]:
d = System(1,2,3)
d.Newthon(1,2,3)

[-0.5000006739940908, -0.9999984345623615, 476.99935296567287]


40000000.0

In [78]:
for i in reversed(range(10)):
    print(i)

9
8
7
6
5
4
3
2
1
0


In [74]:
a

[[0, 0, 0], [0, 0, 0], [2, 2, 2], [3, 3, 3]]

In [75]:
a[2],a[3]=a[3],a[2]

In [76]:
a

[[0, 0, 0], [0, 0, 0], [3, 3, 3], [2, 2, 2]]